In [ ]:
# ipcluster start -n 8 --engines=MPIEngineSetLauncher
import ipyparallel as ipp
client = ipp.Client(profile="default")
%autopx

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline   # with ipyparallel %matplotlib notebook does not work

In [ ]:
import sys
sys.path.append('/home/diego/Projects/Pynterpred_devel/')
import time
import pynterpred as pnt
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from mpi4py import MPI
comm = MPI.COMM_WORLD

In [ ]:
Receptor    = '/home/diego/Projects/Pynterpred_devel/pynterpred/testsystems/Barnase-Barstar/pdbs/Barnase.pdb'
Ligand      = '/home/diego/Projects/Pynterpred_devel/pynterpred/testsystems/Barnase-Barstar/pdbs/Barstar.pdb'

#Receptor    = '/home/diego/Projects/Pynterpred_devel/pynterpred/testsystems/hGH/pdbs/hGH_A.pdb'
#Ligand      = '/home/diego/Projects/Pynterpred_devel/pynterpred/testsystems/hGH/pdbs/hGH_B.pdb'

#Receptor    = '/home/diego/Projects/Pynterpred_devel/pynterpred/testsystems/Fab_5C8/pdbs/Fab_5C8_A.pdb'
#Ligand      = '/home/diego/Projects/Pynterpred_devel/pynterpred/testsystems/Fab_5C8/pdbs/Fab_5C8_B.pdb'

#Receptor    = '/home/diego/Projects/Pynterpred_devel/pynterpred/testsystems/E9034A_ETEC/pdbs/longus_E9034A_ETEC.pdb'
#Ligand      = '/home/diego/Projects/Pynterpred_devel/pynterpred/testsystems/E9034A_ETEC/pdbs/longus_E9034A_ETEC.pdb'

force_field = 'amber14-all.xml'
pH          = 7.0

In [ ]:
docking = pnt.predict(Receptor, Ligand, force_field, pH, delta_x = 1.0, nside = 3,
                      mpi_comm = comm, verbose = True)

In [ ]:
docking.region.rotations

In [ ]:
if comm.Get_rank() == 0:
    import dill as pickle
    with open('docking_10_3.pkl', 'wb') as file:
        pickle.dump(docking, file)

In [ ]:
if comm.Get_rank() == 0:
    xx, pes = docking.get_potential_energy_1D_landscape()

In [ ]:
if comm.Get_rank() == 0:
    %matplotlib inline
    plt.scatter(xx,pes._value,0.1)
    #plt.xlim([-60000,60000])
    plt.ylim([6000,7700])
    plt.xlim([-5000,5000])

In [ ]:
if comm.Get_rank() == 0:
    xx1, pes1 = docking.get_potential_energy_1D_landscape()

In [ ]:
if comm.Get_rank() == 0:
    print(docking.PotentialEnergyNetwork)

In [ ]:
docking = pnt.pynterpred(Barnase, Barstar, force_field, pH, delta_x = 0.5, nside = 5,
                         mpi_comm = comm, verbose = True)

xx1, pes1 = docking.get_potential_energy_1D_landscape()
del(docking)

In [ ]:
docking = pnt.pynterpred(Barnase, Barnase, force_field, pH, delta_x = 0.5, nside = 5, mpi_comm = comm)
xx2, pes2 = docking.get_potential_energy_1D_landscape()
del(docking)

In [ ]:
docking = pnt.pynterpred(Barstar, Barstar, force_field, pH, delta_x = 0.5, nside = 5, mpi_comm = comm)
xx3, pes3 = docking.get_potential_energy_1D_landscape()
del(docking)

In [ ]:
plt.scatter(xx1,pes1,0.1,label="Bb")
plt.scatter(xx2,pes2,0.1,label="BB")
plt.scatter(xx3,pes3,0.1,label="bb")
plt.legend()

plt.xlim([-10000,10000])
plt.ylim([-2000,4000])
